In [1]:
from nbtr.model.hf_model import HfModel
from nbtr.tokenizer.tokenizer import Tokenizer
import torch
from torch.nn import functional as F

In [2]:
model = HfModel.from_pretrained("habanoz/haber-gpt-2-small-v1.0",device="cpu")

number of parameters: 13.77M
Restored model state from repository!


In [3]:
def generate(prompt):
    return model.generate(prompt, max_length = 50)

In [4]:
tokenizer = Tokenizer.from_pretrained("habanoz/news-tr-1.8M-tokenizer-8k")

In [17]:
def generate(model, tokenizer, prompt, max_tokens=50):
    
    prompt = torch.tensor(tokenizer.encode("# Borsa güne yükselişle başladı")['input_ids']).view(1, -1)
    
    for i in range(max_tokens):
        logits = model.forward(prompt)['logits']
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=1)
        tok = torch.multinomial(probs, 1, replacement=True)
        
        if tok == tokenizer.eos_id:
            break

        prompt = torch.cat((prompt , tok), dim=-1)

    text = tokenizer.decode(prompt.tolist()[0])
    print(text)


In [19]:
generate(model, tokenizer, "Borsa güne yükselişle başladı\n")

 ⁇  Borsa güne yükselişle başladı Uyku odasında bulunan 3 bir hafta aranan Muradis, hükümetin 1893'den fazla kötü saat sonra Almanya piyasalarında yaşanan jeopolitik gerilim karşısında yüzde 8-11'le önde kap


In [9]:
tokenizer